In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from neuron.neurons import User, Neuron, ConversationalOrchestrator, ConversationalOrchestratorManager
from neuron.neurons.scripted_users import UserConversationalOrchestrator
import os

In [ ]:
import os
from typing import List

llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

user = User(
    description="You are a human user interacting with conversational and recommendation agents."
)

host_viagens = Neuron(
    name="host_viagens",
    llm_config=llm_config,
    system_message="""
    You are a friendly travel conversation agent.
    Keep the dialogue natural.
    Do not generate travel recommendations.
""",
    description="External agent that manages travel-related conversations."
)

def get_travel_destinations(preference: str) -> List[str]:
    """
    Returns a list of 3 travel destinations based on an exact preference string match.

    Parameters:
        preference (str): A keyword describing the travel preference
                          (e.g., "cultural", "aventura", "gastronomia")

    Returns:
        List[str]: 3 recommended destinations.
    """
    preference = preference.strip().lower()

    if preference == "cultural":
        return ["Lisbon, Portugal", "Rome, Italy", "Kyoto, Japan"]
    elif preference == "aventura":
        return ["Queenstown, New Zealand", "Patagonia, Chile", "Banff, Canada"]
    elif preference == "praia":
        return ["Maui, Hawaii", "Phuket, Thailand", "Fernando de Noronha, Brazil"]
    elif preference == "história":
        return ["Athens, Greece", "Cairo, Egypt", "Jerusalem, Israel"]
    else:
        return ["Paris, France", "New York City, USA", "Tokyo, Japan"]

perfilador = Neuron(
    name="perfilador",
    llm_config=llm_config,
    system_message="""
    Ask short, smart questions to identify the user's travel preferences.
""",
    description="Analyzes user profile and preferences."
)

explainer = Neuron(
    name="explainer",
    llm_config=llm_config,
    system_message="""
    You are an explanation agent specialized in justifying travel destination recommendations.

    Your task is to explain *why* the recommended destinations match the user's preference exactly.
    Base your explanation on the input string provided (e.g., "cultural", "aventura", "praia", etc).

    Be specific and informative. Mention how each destination aligns with the user's interest.

    Do not recommend new places or invent new preferences. Focus only on the reasoning behind the current suggestions.
""",
    description="Explains why the recommended destinations fit the user's stated travel preference."
)


recomenda_viagem = Neuron(
    name="recomenda_viagem",
    llm_config=llm_config,
    system_message="""
    You recommend 3 travel destinations based on user preferences.
    Always use the 'get_travel_destinations' tool. Do not invent destinations.
    """,
    description="Recommends destinations based on preferences.",
    tools=[get_travel_destinations]
)

recomendacao_modulo = ConversationalOrchestrator(
    admin_name="recomendacao_modulo",
    agents=[user, perfilador, recomenda_viagem, explainer],
    messages=[],
    max_round=3,
    speaker_selection_method="auto",
    allow_repeat_speaker=False,
)
modulo_recomendacao = ConversationalOrchestratorManager(
    name="modulo_recomendacao",
    chitchat=recomendacao_modulo,
    llm_config=llm_config,
    description="This module should only be activated when the USER explicitly requests travel recommendations.",
)

chitchat = ConversationalOrchestrator(
    agents=[user, host_viagens, modulo_recomendacao],
    messages=[],
    max_round=100,
    speaker_selection_method="auto",
    allow_repeat_speaker=False,
)

chitchat_manager = ConversationalOrchestratorManager(
    chitchat=chitchat,
    llm_config=llm_config
)

In [4]:
chat_result = user.initiate_chat(
    chitchat_manager, message="Oi tudo bem?",
    cache=None
)
#"Estou planejando minhas férias e queria sugestões de lugares com boa comida e cultura."

user ⟶ conversational_orchestrator:
Oi tudo bem?.

Next speaker: host_viagens

host_viagens ⟶ conversational_orchestrator:
Olá! Tudo bem, obrigado! Você está pensando em viajar para algum lugar em breve?

Next speaker: user

user ⟶ conversational_orchestrator:
quanto é 1+1?

Next speaker: host_viagens

host_viagens ⟶ conversational_orchestrator:
Haha! É 2, é fácil! Você está tentando me testar ou apenas querendo uma pausa na conversa sobre viagens?

Next speaker: user

user ⟶ conversational_orchestrator:
Gostaria de recomendações

Next speaker: modulo_recomendacao


Next speaker: perfilador

perfilador ⟶ modulo_recomendacao:
I'd be happy to provide travel recommendations.

Here's a question to get started:

Are you looking for a relaxing beach vacation or an adventure-filled trip with outdoor activities?

Next speaker: recomenda_viagem

recomenda_viagem ⟶ modulo_recomendacao:
['Queenstown, New Zealand', 'Patagonia, Chile', 'Banff, Canada']
modulo_recomendacao ⟶ conversational_orchestra

In [5]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': 'Oi tudo bem?.', 'role': 'assistant', 'name': 'user'}, {'content': 'Olá! Tudo bem, obrigado! Você está pensando em viajar para algum lugar em breve?', 'name': 'host_viagens', 'role': 'user'}, {'content': 'quanto é 1+1?', 'role': 'assistant', 'name': 'user'}, {'content': 'Haha! É 2, é fácil! Você está tentando me testar ou apenas querendo uma pausa na conversa sobre viagens?', 'name': 'host_viagens', 'role': 'user'}, {'content': 'Gostaria de recomendações', 'role': 'assistant', 'name': 'user'}, {'content': 'Que tipo de viagem você está procurando? Praia, cidade, aventura, comida, cultura? Ou algo mais específico?', 'name': 'modulo_recomendacao', 'role': 'user'}, {'content': 'gostaria de uma explicação', 'role': 'assistant', 'name': 'user'}, {'content': 'Quer dizer que você gostaria de uma recomendação de viagem, mas também gostaria que eu explique por que escolhi aquela opção específica. Isso faz sentido!\n\nVou escolher uma opção e exp